In [40]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [41]:
response = requests.get("https://s02-infrastructure.s3.eu-west-1.amazonaws.com/hackathon-2/website.html")
soup = BeautifulSoup(response.content, 'html.parser')

In [44]:
main_data = soup.find_all('div', class_="main-data")

In [51]:
colnames = [colname.get_text() for colname in main_data[0].find_all("th")]

In [63]:
data = []
for row in main_data[0].find("tbody").find_all("tr"):
    data.append([cell.get_text() for cell in row.find_all("td")])

In [65]:
df = pd.DataFrame(data=data, columns=colnames)

In [67]:
missing_ids = soup.find_all('div', class_="missing-ids")
additional_ids = [x.get_text() for x in missing_ids[0].find("tbody").find_all("td")]

In [77]:
#checking if number of NA ids in main df is the same as length of additional ids
print(np.sum(df.id=="<NA>"), len(additional_ids))

999 999


In [90]:
for index, additional_id in enumerate(additional_ids):
    df.iloc[index].id=additional_id

In [93]:
#check if all ids have been successfully found
np.sum(df.id=="<NA>")

0

In [115]:
df = df.astype('int64')
df.head()

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,...,ExtFavicon,InsecureForms,RelativeFormAction,ExtFormAction,AbnormalFormAction,PctNullSelfRedirectHyperlinks,FrequentDomainNameMismatch,FakeLinkInStatusBar,RightClickDisabled,CLASS_LABEL
0,7368,1,0,3,116,0,0,0,0,3,...,1,1,0,0,0,0,0,0,0,0
1,3881,4,1,2,44,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,1
2,5393,2,0,2,108,0,0,0,0,2,...,0,1,0,0,0,0,0,0,0,0
3,1401,3,1,2,40,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,4925,2,1,3,42,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [117]:
df.to_csv("webscraping.csv")